In [ ]:
import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

import sys

IN_COLAB = 'google.colab' in sys.modules
REPO_DIR = '..' if IN_COLAB  else '..'

!git clone https://github.com/google-research/graph-attribution.git --quiet
    
import sys
sys.path.insert(1, '/kaggle/working/graph-attribution')

!pip install tensorflow tensorflow-probability -q
!pip install dm-sonnet -q
!pip install graph_nets "tensorflow>=2.1.0-rc1" "dm-sonnet>=2.0.0b0" tensorflow_probability
!pip install git+https://github.com/google-research/graph-attribution -quiet
    
!pip install git+https://github.com/google-research/graph-attribution

In [ ]:
import os
import itertools
import collections
import tqdm.auto as tqdm

from IPython.display import display

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
import sonnet as snt
import graph_nets
from graph_nets.graphs import GraphsTuple
import graph_attribution as gatt

from tqdm import tqdm

import time

import networkx as nx

import warnings
warnings.simplefilter("ignore", UserWarning)

for mod in [tf, snt, gatt]:
    print(f'{mod.__name__:20s} = {mod.__version__}')
    
from graph_attribution import tasks
from graph_attribution import graphnet_models as gnn_models
from graph_attribution import graphnet_techniques as techniques
from graph_attribution import datasets
from graph_attribution import experiments
from graph_attribution import templates
from graph_attribution import graphs as graph_utils

#datasets.DATA_DIR = os.path.join(REPO_DIR, 'data')
#print(f'Reading data from: {datasets.DATA_DIR}')

datasets.DATA_DIR = './graph-attribution/data'

task_type = 'logic7'
block_type = 'gcn'

#task_dir = datasets.get_task_dir(task_type)
task_dir = './graph-attribution/data/logic7'
exp, task, methods = experiments.get_experiment_setup(task_type, block_type)
task_act, task_loss = task.get_nn_activation_fn(), task.get_nn_loss_fn()
graph_utils.print_graphs_tuple(exp.x_train)
print(f'Experiment data fields:{list(exp.__dict__.keys())}')

In [ ]:
# Source: https://notebook.community/deepmind/graph_nets/graph_nets/demos/graph_nets_basics
graphs_nx = graph_nets.utils_np.graphs_tuple_to_networkxs(exp.x_test)

def nx_g_plotter(graphs_nx, ColNum=8, node_clr='#ff8080'):
    _, axs = plt.subplots(ncols=ColNum, nrows = 1, figsize=(30, 5))
    for iax, (graph_nx2, ax) in enumerate(zip(graphs_nx, axs)):
        nx.draw(graph_nx2, ax=ax, node_color=node_clr)
        ax.set_title("Graph {}".format(iax))

In [ ]:


graphs_nx_1 = []
graphs_nx_0 = []

for ii, g_net_ii in enumerate(graphs_nx):
    if exp.y_train[ii] == 1:
        graphs_nx_1.append(g_net_ii)
    else:
        graphs_nx_0.append(g_net_ii)
        
nx_g_plotter(graphs_nx_1, ColNum=8, node_clr='#ff8080')
nx_g_plotter(graphs_nx_0, ColNum=8, node_clr='#00bfff')